In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
from scipy.stats import chi2
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr, spearmanr
import pandas.core.algorithms as algos
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec 
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
max_bin = 20
force_bin = 3
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve, plot_confusion_matrix, classification_report
from IPython.display import Image
import pydotplus
import matplotlib.pyplot as plt
%matplotlib inline
color = sns.color_palette()
seed = 42
!pip install autoimpute
!pip install fancyimpute

from sklearn.impute import SimpleImputer
import time
import gc

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Essay/Data/final_data1.csv')
df.head()

,Unnamed: 0,index,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR
0,0,0,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(356251, 526)

In [ ]:
df.isnull().sum()

Unnamed: 0                               0
index                                    0
TARGET                               48744
CODE_GENDER                              0
FLAG_OWN_CAR                             0
                                     ...  
CC_NAME_CONTRACT_STATUS_nan_MIN     252693
CC_NAME_CONTRACT_STATUS_nan_MAX     252693
CC_NAME_CONTRACT_STATUS_nan_MEAN    252693
CC_NAME_CONTRACT_STATUS_nan_SUM     252693
CC_NAME_CONTRACT_STATUS_nan_VAR     253385
Length: 526, dtype: int64

In [ ]:
def fill_missing(df): 
  column = [col for col in df.columns]
  df[column] = df[column].fillna(df[column].mode().iloc[0])
  return df

fill_missing(df)

,Unnamed: 0,index,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CC_NAME_CONTRACT_STATUS_Sent proposal_MAX,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR
0,0,0,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356246,356250,48739,0.0,1,0,0,0,121500.0,412560.0,17473.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
356247,356251,48740,0.0,1,0,1,2,157500.0,622413.0,31909.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
356248,356252,48741,0.0,1,1,0,1,202500.0,315000.0,33205.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
356249,356253,48742,0.0,0,0,1,0,225000.0,450000.0,25128.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df = df.drop('Unnamed: 0', axis = 1)

In [ ]:
#Fill missing data

def feature_type_split(data, special_list=[]):
    cat_list = []
    dis_num_list = []
    num_list = []
    for i in data.columns.tolist():
        if data[i].dtype == 'object':
            cat_list.append(i)
        elif data[i].nunique() < 25:
            dis_num_list.append(i)
        elif i in special_list:     # if you want to add some special cases
            dis_num_list.append(i)
        else:
            num_list.append(i)
    return cat_list, dis_num_list, num_list


def missing_value(df, cat_list,num_list):
  start = time.time()
  df[cat_list] = SimpleImputer(strategy='most_frequent').fit_transform(df[cat_list])
  df[dis_num_list] = SimpleImputer(strategy='most_frequent').fit_transform(df[dis_num_list])
  end = time.time()
  print('discrete imputing time:', str(round((end - start)/60)), 'mins')
  # # continuous 
  from fancyimpute import IterativeImputer as MICE
  start = time.time()
  df[num_list] = MICE(initial_strategy='median').fit_transform(df[num_list])
  end = time.time()
  print('continuous imputing time:', str(round((end - start)/60)), 'mins')


#resampling
# umbalanced data
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
def over_sampling(X, y):
  ros = RandomOverSampler(random_state=25)
  X_ros, y_ros = ros.fit_resample(X, y)
  return X_ros, y_ros 
def under_sampling(X, y):
  rus = RandomUnderSampler(random_state = 42, replacement = True)
  X_rus, y_rus = rus.fit_resample(X, y)
  return X_rus, y_rus
def smothing(X, y):
  smote = SMOTE()
  X_smote, y_smote = smote.fit_resample(X, y)
  return X_smote, y_smote

#Modeling
#Chia data
from sklearn.model_selection import train_test_split
def split_data(X,y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val 

#Scaling data (Tìm hiểu xem có những cách scaling nào)
from sklearn.preprocessing import StandardScaler
def scale(X_train, X_val):
    sc_X = StandardScaler()
    X_train = sc_X.fit_transform(X_train)
    X_val = sc_X.transform(X_val)
    return X_train, X_val

from sklearn.preprocessing import MinMaxScaler
def scale_minmax(X_train, X_val):
  scaler_minmax = MinMaxScaler()
  X_train = scaler_minmax.fit_transform(X_train)
  X_val = scaler_minmax.transform(X_val)
  return X_train, X_val


# Model
def metrics(model, X_val, y_val):
  y_pred = model.predict(X_val)
  ypred_prob = model.predict_proba(X_val)[:,1]
  target_names = ["Class 0", "Class 1"]
  print("AUC",roc_auc_score(y_val, ypred_prob))
  print(classification_report(y_val, y_pred, target_names=target_names))
  return None
  
#print("Classification Accuracy:", round(accuracy_score(y_val, y_pred)*100,2)) 
#print("Precision_score is", round(precision_score(y_val, y_pred)*100,2))
#print("Recall score", round(recall_score(y_val,y_pred)*100,2))
#print("F1 score: ", round(f1_score(y_val,y_pred, average = "weighted") * 100,2))

## Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

def log(X_train,y_train,X_val, y_val):
  log = LogisticRegression()
  model = log.fit(X_train, y_train)
  y_predict = log.predict(X_val)
  a = metrics(model,X_val,y_val)
  return model
  

 


## Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

def model_random_forest(X_train, y_train, X_val, y_val): 
  clf = RandomForestClassifier(criterion="gini",\
                               random_state = 25,\
                               min_samples_leaf=5)
  model = clf.fit(X_train, y_train)
  y_predict = clf.predict(X_val)
  a = metrics(model,X_val,y_val)
  return model

def rf(X_train,y_train,X_val):
  n_estimators = [5,20,50,100] 
  max_features = ['auto', 'sqrt'] # number of features in consideration at every split
  max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
  min_samples_split = [2, 6, 10] # minimum sample number to split a node
  min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
  bootstrap = [True, False] # method used to sample data points

  random_grid = {'n_estimators': n_estimators,'max_features': max_features,'max_depth': max_depth,'min_samples_split': min_samples_split,'min_samples_leaf': min_samples_leaf,'bootstrap': bootstrap}
  from sklearn.ensemble import RandomForestRegressor
  rf = RandomForestRegressor()
  from sklearn.model_selection import RandomizedSearchCV
  rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)
  rf_random.fit(X_train, y_train)
  print('Best Parameters: ', rf_random.best_params_, ' \n')
  

  #randmf = RandomForestRegressor(n_estimators = 100, min_samples_split = 6, min_samples_leaf= 4, max_features = 'sqrt', max_depth= 120, bootstrap=False) 
  #randmf.fit( X_train, y_train)
  #y_pred = randmf.predict(X_val)
  #ypred_prob = randmf.predict_proba(X_val)[:, 1]
  #print(' AUC : %.6f' % ( roc_auc_score(y_val, ypred_prob)))
  #plot_AUC( y_val, ypred_prob)
  return None


## XGB
def xgb(X_train, y_train, X_val, y_val):
  import xgboost as xgb
  param_init = {
    "objective": "binary:logistic", 
    "booster": "gbtree", 
    "max_depth": 4, # default: 3 only for depthwise
    "n_estimators": 1000, # default: 500  
    "learning_rate": 0.025, # default: 0.05 
    "subsample": 0.7, 
    "colsample_bytree": 0.6,  # default:  1.0
    "colsample_bylevel": 0.5, # default: 1.0
    "random_state": 0,
    
    #
    "silent": True, 
    "n_jobs": 16, 
    
    #
    "tree_method": "hist", # default: auto
    "grow_policy": "lossguide" # default depthwise
    }
    
  param_fit = {
    "eval_metric": "auc", 
    "early_stopping_rounds": 500, # default: 100
    "verbose": 200,
    "eval_set": [(X_train, y_train), (X_val, y_val)]}

  xgb_model = xgb.XGBClassifier(**param_init)
  xgb_model.fit(X_train, y_train, **param_fit)
  evals_result = xgb_model.evals_result()


#lightgbm
def model_LGBM(X_train, y_train, X_val, y_val): 
  clf = lgb.LGBMClassifier(boosting_type = 'goss',
            nthread=4,
            n_estimators=10000,
            learning_rate=0.005134,
            num_leaves=54,
            colsample_bytree=0.508716,
            subsample=1,
            max_depth=10,
            reg_alpha=0.436193,
            reg_lambda=0.479169,
            min_split_gain=0.024766,
            min_child_weight=40,
            silent=-1,
            verbose=-1,
            is_unbalance=False)
  model = clf.fit(X_train, y_train)
  a = metrics(model,X_val,y_val)
  return model



##Chia Kfold - tuning lightGBM
### dùng GridSearchCV để chọn param



### Chia K-fold - tuning Catboots


#Metrics Performance
## Đánh giá mô hình thông qua các metrics.
import numpy as np
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve, plot_confusion_matrix


##AUC - ROC
def plot_AUC( y_val, ypred_prob):
    from sklearn import metrics
    fig, (ax, ax1) = plt.subplots(nrows = 1, ncols = 2, figsize = (15,5))
    
    fpr, tpr, threshold = metrics.roc_curve(y_val, ypred_prob)
    roc_auc = metrics.auc(fpr, tpr)

    ax.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    ax.plot([0, 1], [0, 1],'r--')
    ax.set_title('Receiver Operating Characteristic ',fontsize=10)
    ax.set_ylabel('True Positive Rate',fontsize=20)
    ax.set_xlabel('False Positive Rate',fontsize=15)
    ax.legend(loc = 'lower right', prop={'size': 16})
    plt.subplots_adjust(wspace=1)

In [ ]:
np.any(np.isnan(df))

False

In [ ]:
np.all(np.isfinite(df))

False

In [ ]:
df = df.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356251 entries, 0 to 356250
Columns: 525 entries, index to CC_NAME_CONTRACT_STATUS_nan_VAR
dtypes: float64(358), int64(167)
memory usage: 1.4 GB


In [ ]:
df.shape

(356251, 525)

In [ ]:
#Split
y = df.TARGET
X = df.drop('TARGET', axis = 1)
X_train, X_val, y_train, y_val = split_data(X,y)
scale(X_train, X_val)

#Sampling
X_ros, y_ros = over_sampling(X_train, y_train)
X_rus, y_rus = under_sampling(X_train, y_train)
X_smooth, y_smooth = smothing(X_train, y_train)

In [ ]:
print("Oversampling:")
model1 = log(X_ros, y_ros, X_val, y_val)
print("\nUndersampling:")
model2 = log(X_rus, y_rus, X_val, y_val)
print("\nSmote: ")
model3 = log(X_smooth, y_smooth, X_val, y_val)

In [ ]:
print("Oversampling:")
model1 = model_random_forest(X_ros, y_ros, X_val, y_val)
print("\nUndersampling:")
model2 = model_random_forest(X_rus, y_rus, X_val, y_val)
print("\nSmote: ")
model3 = model_random_forest(X_smooth, y_smooth, X_val, y_val)

In [ ]:
print("Oversampling:")
model1 = model_LGBM(X_ros, y_ros, X_val, y_val)
print("\nUndersampling:")
model2 = model_LGBM(X_rus, y_rus, X_val, y_val)
print("\nSmote: ")
model3 = model_LGBM(X_smooth, y_smooth, X_val, y_val)